In [2]:
# default libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
#import seaborn as sns
#sns.set()

# for data preprocessing
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

In [3]:
income = pd.read_csv("./adult.csv")
income.head()

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K
4,18,?,103497,Some-college,10,Never-married,?,Own-child,White,Female,0,0,30,United-States,<=50K


In [4]:
def init_check(df):
    """
    A function to make initial check for the dataset including the name, data type, 
    number of null values and number of unique varialbes for each feature.
    
    Parameter: dataset(DataFrame)
    Output : DataFrame
    """
    columns = df.columns    
    lst = []
    for feature in columns : 
        dtype = df[feature].dtypes
        num_null = df[feature].isnull().sum()
        num_unique = df[feature].nunique()
        lst.append([feature, dtype, num_null, num_unique])
    
    check_df = pd.DataFrame(lst)
    check_df.columns = ['feature','dtype','num_null','num_unique']
    check_df = check_df.sort_values(by='dtype', axis=0, ascending=True)
    
    return check_df

In [5]:
init_check(income)

,feature,dtype,num_null,num_unique
0,age,int64,0,74
2,fnlwgt,int64,0,28523
4,educational-num,int64,0,16
10,capital-gain,int64,0,123
11,capital-loss,int64,0,99
12,hours-per-week,int64,0,96
1,workclass,object,0,9
3,education,object,0,16
5,marital-status,object,0,7
6,occupation,object,0,15


In [6]:
def categorical_encoding(df, categorical_cloumns, encoding_method):
    """
    A function to encode categorical features to a one-hot numeric array (one-hot encoding) or 
    an array with value between 0 and n_classes-1 (label encoding).
    
    Parameters:
        df (pd.DataFrame) : dataset
        categorical_cloumns  (string) : list of features 
        encoding_method (string) : 'one-hot' or 'label'
    Output : pd.DataFrame
    """
    
    if encoding_method == 'label':
        print('You choose label encoding for your categorical features')
        encoder = LabelEncoder()
        encoded = df[categorical_cloumns].apply(encoder.fit_transform)
        return encoded
    
    elif encoding_method == 'one-hot':
        print('You choose one-hot encoding for your categorical features') 
        encoded = pd.DataFrame()
        for feature in categorical_cloumns:
            dummies = pd.get_dummies(df[feature], prefix=feature)
            encoded = pd.concat([encoded, dummies], axis=1)
        return encoded

In [7]:
def data_preprocessing(df, features, target, encoding_method, test_size, random_state):
    
    
    y = df[target]
    
    X = df[features]
    
    categorical_columns = X.select_dtypes(include=['object']).columns
    
    if len(categorical_columns) != 0 :
        encoded = categorical_encoding(X, categorical_cloumns=categorical_columns, encoding_method=encoding_method)
        X = X.drop(columns=categorical_columns, axis=1)
        X = pd.concat([X, encoded], axis=1)
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)
    
    scaler=MinMaxScaler()
    X_train= pd.DataFrame(scaler.fit_transform(X_train))
    X_test = pd.DataFrame(scaler.transform(X_test))
    
    return X_train, X_test, y_train, y_test

In [8]:
features = income.columns.drop('income')
X_train, X_test, y_train, y_test = data_preprocessing(df=income, features=features, 
                                                      target='income', encoding_method = 'label',
                                                      test_size=0.2, random_state=0)

print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

You choose label encoding for your categorical features
(39073, 14) (9769, 14) (39073,) (9769,)


In [12]:
y_train = y_train.replace('<=50K',0)
y_train = y_train.replace('>50K',1)
y_train

22729    0
8650     0
26605    0
15864    0
2516     0
        ..
21243    0
45891    0
42613    1
43567    0
2732     1
Name: income, Length: 39073, dtype: int64

In [13]:
y_test = y_test.replace('<=50K',0)
y_test = y_test.replace('>50K',1)
y_test

38113    0
39214    1
44248    1
10283    0
26724    0
        ..
27608    1
25140    0
18962    0
36301    0
45837    0
Name: income, Length: 9769, dtype: int64

In [14]:
X_train.to_csv('X_train.csv', index=False)
X_test.to_csv('X_test.csv', index=False)
y_train.to_csv('y_train.csv', index=False)
y_test.to_csv('y_test.csv', index=False)